<a href="https://colab.research.google.com/github/rhaymisonbetini/bert-model-from-scratch/blob/main/ARQUITETURA_BERT_FROM_SCRATCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**BERT MODEL FROM SCRATCH**

In [1]:
import random
random.seed(10)

In [2]:
import re
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import *

In [11]:
text = open('frases.txt','r').read()

In [151]:
sentences = re.sub("['.,!?\\n-]",' ', text.lower()).split('\\n')
word_list = list(set("".join(sentences).split()))
word_dict = {'[PAD]':0, '[CLS]':1, '[SEP]':2,'[MASK]':3}

for i, w in enumerate(word_list):
  word_dict[w] = i+4

number_dict = {i:w for i, w in enumerate(word_dict)} # colocando os indices como chave e as palavras como valor no dicionario
vocab_size = len(word_dict)
token_list = list()

for sentence in sentences:
  arr = [word_dict[s] for s in sentence.split()]
  token_list.append(arr)

In [154]:
#hiperparametros
batch_size =6
n_segments = 2
dropout = 0.2

#comprimento maximo
maxlen = 100

max_pred = 7 #numero de palavras a serem preditas

n_layers = 6 #numero de camadas

n_heads = 12 #numero de cabeças de mult head attention

d_model = 768 #tamanho do embedding

d_ff = d_model * 4 #tamanho do feed forward

d_k = 64 # Dimensao de K(=Q)V

d_v = 64 # Dimensao de K(=Q)V

NUM_EPOCHS = 50 #numero de epocas

CRIANDO **BATCHES**

In [155]:
# Função para criar os batches de dados
def make_batch():

    batch = []

    positive = negative = 0

    while positive != batch_size/2 or negative != batch_size/2:

        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))

        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]

        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]

        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        # MASK LM (MLM) de 15 % dos tokens em uma sentença
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15))))

        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]

        shuffle(cand_maked_pos)

        masked_tokens, masked_pos = [], []

        for pos in cand_maked_pos[:n_pred]:

            masked_pos.append(pos)

            masked_tokens.append(input_ids[pos])

            if random() < 0.8:
                input_ids[pos] = word_dict['[MASK]']
            elif random() < 0.5:
                index = randint(0, vocab_size - 1)
                input_ids[pos] = word_dict[number_dict[index]]

        # Zero Paddings
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:

            # IsNext
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True])
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:

            # NotNext
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False])
            negative += 1

    return batch

**ATTENTION MASK**

In [156]:
# Função para o padding
def get_attn_pad_masked(seq_q, seq_k):

    batch_size, len_q = seq_q.size()

    batch_size, len_k = seq_k.size()

    pad_attn_masked = seq_k.data.eq(0).unsqueeze(1)

    return pad_attn_masked.expand(batch_size, len_q, len_k)

In [157]:
batch = make_batch()

In [159]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

In [ ]:
get_attn_pad_masked(input_ids, input_ids)[0][0], input_ids[0]